In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
import InbeddingDiagramDNeg as Dia
import WormholeRayTracer as wrmhole
import WormholeGraphics as wg
import Symplectic_DNeg as Smpl

In [3]:
path = os.getcwd()

In [4]:
Par = [0.8/1.42953, 8.6, 4.3] # M, rho, a parameters wormhole

initial_q = np.array([12, np.pi, np.pi/2])

In [5]:
print(initial_q)
Grid_dimension = '3D'
mode = 0
Motion1, Grid1, CM1 = wrmhole.Simulate_DNeg(Smpl.Sympl_DNeg, Par, 0.05, 2000, initial_q, 20**2, 20**2, Grid_dimension, mode, wg.Grid_constr_3D_Sph)

[12.          3.14159265  1.57079633]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
Cosyst =  ' Sph '
cv2.imwrite(os.path.join(path, "Pictures/Image "+Grid_dimension+"Gr"+Cosyst+"DNeg Sympl"+str(Par)+" "+str(initial_q)+".png"),
            255*wg.Make_Pict_RGBP(Motion1[-1, 1], Grid1))

In [ ]:
cv2.imwrite(os.path.join(path, "Pictures/Image "+"2DGr DNeg Sympl"+str(Par)+" "+str(initial_q)+".png"),
            255*wg.Make_Pict_RGBP(Motion1[-1, 1], wg.Grid_constr_2D(Motion1[-1, 1], 11, 1, 0.05)))

In [ ]:
cv2.imwrite(os.path.join(path, "Pictures/Image "+"NoGrid DNeg Sympl"+str(Par)+" "+str(initial_q)+".png"),
            255*wg.Make_Pict_RGBP(Motion1[-1, 1], np.zeros(Motion1[-1, 1, 0].shape, dtype=bool)))

In [ ]:
Motion2 = wrmhole.simulate_raytracer_fullpath(100, Par, initial_q, 20**2, 20**2)

In [ ]:
cv2.imwrite(os.path.join(path, "Pictures/Image "+"2DGr DNeg Scipy intgr."+str(Par)+" "+str(initial_q)+".png"),
            255*wg.Make_Pict_RGBP(Motion2[-1, 1], wg.Grid_constr_2D(Motion2[-1, 1], 11, 1, 0.05)))

In [ ]:
Geo_label = None #list of strings for labeling geodesics, turn Geo_sel to None
#Geo_Sel = None
Geo_Sel = [[31,29],[64,54],[99,31],[97,52],[146,146],[203,39]]
if Geo_Sel == None:
    Geo_txt = ""
else:
    Geo_txt = str(Geo_Sel)
wg.gdsc(Motion1, Par, "Pictures/geodesics "+Geo_txt+" DNeg Sympl"+str(Par)+" "+str(initial_q)+".png", path, Geo_label, Geo_Sel)